In [1]:
%load_ext autoreload
%load_ext autotime

time: 196 µs (started: 2024-10-08 18:01:30 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 579 µs (started: 2024-10-08 18:01:30 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 819 µs (started: 2024-10-08 18:01:30 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-cosmoflow-dlio-v100-node-4-v1"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/cosmoflow-dlio-v100-node-4-v1"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cosmoflow/dlio-v100/node-4/v1/RAW/*.pfw.gz"


time: 697 µs (started: 2024-10-08 18:01:30 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 8.68 s (started: 2024-10-08 18:01:30 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 17min 49s (started: 2024-10-08 18:01:39 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          1179.61 seconds                                                                               │
│  I/O Time         56.68 seconds                                                                                 │
│                   ├── Read - 53.77 seconds (94.87%)                                                             │
│                   ├── Write - 0.01 seconds (0.01%)                                                              │
│                   └── Metadata - 3.20 seconds (5.64%)                                                           │
│  I/O Operations   39,207,478 ops                                                                                │
│                   ├── Read - 31,856,216 ops (81.25%)                                                            │
│                   ├── Write - 1,855 ops (0.00%)                                                                 │
│                   └── Metadata - 7,340,575 ops (18.72%)                                                         │
│  I/O Size         6.74 TiB                                                                                      │
│                   ├── Read - 6.74 TiB (100.00%)                                                                 │
│                   └── Write - 215.48 kiB (0.00%)                                                                │
│  Read Requests    4 kiB-256 kiB - 31,855,638 ops                                                                │
│                   ├── <4 kiB - 43,936 ops (0.14%)                                                               │
│                   ├── 4-16 kiB - 5,088 ops (0.02%)                                                              │
│                   ├── 16-64 kiB - 7,980 ops (0.03%)                                                             │
│                   └── 64-256 kiB - 31,798,634 ops (99.82%)                                                      │
│  Write Requests   4 kiB-4 kiB - 1,855 ops                                                                       │
│                   └── <4 kiB - 1,855 ops (100.00%)                                                              │
│  Nodes            4 nodes                                                                                       │
│                   ├── 14200274195875936004 - 56.68 s (100.00%) - 1.69 TiB/183.79 kiB R/W (25.00/0.00%) -        │
│                   │   9,802,590 ops (25.00%)                                                                    │
│                   ├── 17424505939907203644 - 56.61 s (99.88%) - 1.69 TiB/10.56 kiB R/W (25.00/0.00%) -          │
│                   │   9,801,486 ops (25.00%)                                                                    │
│                   ├── 12775585946864612109 - 56.25 s (99.25%) - 1.69 TiB/10.56 kiB R/W (25.00/0.00%) -          │
│                   │   9,801,058 ops (25.00%)                                                                    │
│                   └── 16317731584986882421 - 54.87 s (96.82%) - 1.69 TiB/10.56 kiB R/W (25.00/0.00%) -          │
│                       9,802,344 ops (25.00%)                                                                    │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  3,091 processes                                                                               │
│  Files            553,096 files                                                                                 │
│                   ├── Shared: 529,138 files (95.67%)                                                            │
│                   └── FPP: 23,958 files (4.33%)       

time: 6min 55s (started: 2024-10-08 18:19:29 -07:00)
